# LiverTox parser

LiverTox has been moved from Toxnet to NLM bookshelf with a public domain licensing. It has not been retired and is still updated. NLM bookshelf has an ftp server for retrieving open-licensed materials.  In the process of migrating LiverTox over, it has also been more strictly formatted, making it slightly more easy to parse for information.

To obtain the latest dataset, pull the ftp index from here:
ftp://ftp.ncbi.nlm.nih.gov/pub/litarch/file_list.txt

Next, search for LiverTox and get the url. Every entry in livertox is stored as a pdf and xml file in the compressed file. The filename is the livertox urlstub. The urlbase for LiverTox is https://www.ncbi.nlm.nih.gov/books/n/livertox/


In [2]:
from wikidataintegrator import wdi_core, wdi_login, wdi_helpers
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs
import pandas as pd
from pandas import read_csv
import requests
import ftplib
from tqdm.notebook import trange, tqdm
import ipywidgets 
import widgetsnbextension
import xml.etree.ElementTree as et 
import time
import os


In [ ]:
## Note that the property start date is used for list date.
## When placed in the references, Deltabot moved it out as a qualifier

from datetime import datetime
import copy
def create_reference(prop65_url):
    refStatedIn = wdi_core.WDItemID(value="Q28455381", prop_nr="P248", is_reference=True)
    timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    refURL = wdi_core.WDUrl(value=prop65_url, prop_nr="P854", is_reference=True)
    return [refStatedIn, refRetrieved, refURL]

In [ ]:
print("Logging in...")
import wdi_user_config ## Credentials stored in a wdi_user_config file
login_dict = wdi_user_config.get_credentials()
login = wdi_login.WDLogin(login_dict['WDUSER'], login_dict['WDPASS'])


In [9]:
urls_dict = {
    'ftp_index':'ftp://ftp.ncbi.nlm.nih.gov/pub/litarch/file_list.txt',
    'ftp_base':'ftp://ftp.ncbi.nlm.nih.gov/pub/litarch/',
    'bookshelf_base':'https://www.ncbi.nlm.nih.gov/books/n/livertox/',
    'local_data':'data/'
    }

In [12]:
## Fetch the index file to determine the url of the current livertox dataset
#bookshelf_index = ftplib.FTP(urls_dict['ftp_base'])
#bookshelf_index.login()

#print(bookshelf_index.text)

## This is an example of how to use the ftplib to pull things from an ftp server
"""
import os
from ftplib import FTP
 
ftp = FTP("www.myWebsite.com", "USERNAME", "PASSWORD")
ftp.login()
ftp.retrlines("LIST")
 
ftp.cwd("folderOne")
ftp.cwd("subFolder") # or ftp.cwd("folderOne/subFolder")
 
listing = []
ftp.retrlines("LIST", listing.append)
words = listing[0].split(None, 8)
filename = words[-1].lstrip()
 
# download the file
local_filename = os.path.join(r"c:\myfolder", filename)
lf = open(local_filename, "wb")
ftp.retrbinary("RETR " + filename, lf.write, 8*1024)
lf.close()
"""

In [14]:
## Once the file has been downloaded, process it for data of interest
import tarfile
import tempfile
import stat

tar_file = urls_dict['local_data']+'livertox_NBK547852.tar.gz'
tar = tarfile.open(tar_file, "r:gz")
members = tar.getmembers()
print(len(members))

3372


In [221]:
phenotypes = []
drug_types = []
misc_pages = []
drug_page = []
for member in members[11:15]:
    if '.nxml' in str(member):
        extracted = tar.extractfile(member)
        tree = et.parse(extracted)
        root = tree.getroot()
        try:
            metainfo = root.find('book-part').find('book-part-meta')
            url_stub = metainfo.find('book-part-id').text
            drug_name = metainfo.find('title-group').find('title').text
            date_info = metainfo.find('pub-history').find('date')
            date_day = date_info.find('day').text
            date_month = date_info.find('month').text
            date_year = date_info.find('year').text
            tmp_date = str(date_year)+"-"+str(date_month)+"-"+str(date_day)
            if date_info.attrib['date-type'] == 'updated':
                update_date = tmp_date
                original_date = None
            else:
                original_date = tmp_date
                update_date = None
                
            basic_meta = {'Title':drug_name,'url_stub':url_stub,
                          'last_update':update_date,'original_date':original_date}
            ## Determine if the page is about a drug, drug class, or phenotype
            if 'Phenotypes' in url_stub:
                phenotypes.append(basic_meta)
            else:
                book_content = root.find('book-part').find('body').find('sec')
                ## We're only interested in the Overview section of the drug and drug class pages
                if ".OVERVIEW" in book_content.attrib['id']:
                    print(url_stub)
                    basic_tags = [elem.tag for elem in book_content.iter()]
                    if 'list' in basic_tags:
                        drug_types.append(basic_meta)
                        print('listed')
                    else:
                        id_to_find = url_stub+".Hepatotoxicity"
                        print(hepatox_sxn)    
                        
                        """
                        Need to find a way to pull section of interest for parsing
                        section = book_content.find('sec')
                        if section.attrib['id']==url_stub+".Hepatotoxicity":
                            overview_text = section.text
                            score_start = overview_text.find('Likelihood score: ')
                            score_end = overview_text[score_start:len(overview_text)].find('</p>')
                            likelihood_score = overview_text[score_start:score_start+score_end]
                            print(likelihood_score)
                        ## if the page has a Hepatoxicity section, it's likely a drug 

                            ## the page is a drug page, so we can parse for toxicity scores"""
                                  
        except:
            misc_pages.append(str(root))
            #print(member)

Quinine
Leflunomide
Propranolol


In [175]:
print(drug_types)


[]
